# Практическая работа № 5 «Тайные многосторонние вычисления и разделение секрета»
 
В практической работе необходимо привести последовательность выполнения следующих протоколов:
- тайных многосторонних вычислений для расчета средней величины трех чисел. В качестве исходных данных принять коды 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите;
- разбиения секрета с использование гаммирования для трех участников. В качестве секрета принять первые 3 буквы фамилии, для гамм - любые трехбуквенные сочетания;
- разделения секрета по схеме Шамира для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите;
- разделения секрета по схеме Асмута-Блума для (3, 5)-пороговой схемы. В качестве секрета S принять код 1-ой буквы своей фамилии согласно ее положению в алфавите.

При оформлении отчета необходимо привести исходные данные и таблицы, содержащие последовательность выполнения протоколов. Отчет необходимо загрузить в edu.susu.ru


In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
with open('instance/settings.json') as f:
    settings = json.load(f)
    LAST_NAME = settings['last_name']
    FIRST_NAME = settings['first_name']
    PATRONYMIC = settings['patronymic']

In [28]:
def power_with_modulo(a, n, m):
    res = 1
    while n > 0:
        if (n & 1) == 1:
            res = (res * a) % m
        a = (a * a) % m
        n >>= 1
    return res

## Тайные многосторонние вычисления

In [21]:
def rsa_generator():
    keys = [
        [5, 91, 29],
        [7, 91, 31],
        [17, 91, 17],
    ]
    for e, n, d in keys:
        yield {'public': {'e': e, 'n': n}, 'private': {'d': d}}

In [23]:
smc_data = list(map(lambda c: ord(c) - ord('a') + 1, LAST_NAME[:3]))
smc_data = {k: {'secret': v, 'rsa': rsa} for k, v, rsa in zip('abc', smc_data, rsa_generator())}
print(json.dumps(smc_data, indent=4))

{
    "a": {
        "secret": 1,
        "rsa": {
            "public": {
                "e": 5,
                "n": 91
            },
            "private": {
                "d": 29
            }
        }
    },
    "b": {
        "secret": 12,
        "rsa": {
            "public": {
                "e": 7,
                "n": 91
            },
            "private": {
                "d": 31
            }
        }
    },
    "c": {
        "secret": 11,
        "rsa": {
            "public": {
                "e": 17,
                "n": 91
            },
            "private": {
                "d": 17
            }
        }
    }
}


<b>A</b> прибавляет случайное секретное число $x$ к своему секрету, шифрует результат с помощью открытого ключа <b>B</b> и отсылает его <b>B</b>

In [68]:
np.random.seed(42)
a_x = np.random.randint(2, 1<<20)

a_data = smc_data['a']['secret'] + a_x
shared_data = power_with_modulo(
    a_data, 
    smc_data['b']['rsa']['public']['e'], 
    smc_data['b']['rsa']['public']['n'],
)

{{f'$secret_a = {smc_data["a"]["secret"]}$'}}  
{{f'$x_a = {a_x}$'}}  
{{f'$data_a = {a_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

<b>B</b> расшифровывает результат, добавляет к нему свою зарплату, шифрует результат с помощью открытого ключа <b>C</b> и отсылает его <b>C</b>.

In [69]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['b']['rsa']['private']['d'], 
    smc_data['b']['rsa']['public']['n'],
)
b_data = decrypted_data + smc_data['b']['secret']
shared_data = power_with_modulo(
    b_data, 
    smc_data['c']['rsa']['public']['e'], 
    smc_data['c']['rsa']['public']['n'],
)

{{f'$secret_b = {smc_data["b"]["secret"]}$'}}  
{{'$data_{decrypted} = %d$' % decrypted_data}}  
{{f'$data_b = {b_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

In [70]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['c']['rsa']['private']['d'], 
    smc_data['c']['rsa']['public']['n'],
)
c_data = decrypted_data + smc_data['c']['secret']
shared_data = power_with_modulo(
    c_data, 
    smc_data['a']['rsa']['public']['e'], 
    smc_data['a']['rsa']['public']['n'],
)

{{f'$secret_c = {smc_data["c"]["secret"]}$'}}  
{{'$data_{decrypted} = %d$' % decrypted_data}}  
{{f'$data_c = {c_data}$'}}   
{{f'$shared\\_data = {shared_data}$'}}

In [73]:
decrypted_data = power_with_modulo(
    shared_data, 
    smc_data['a']['rsa']['private']['d'], 
    smc_data['a']['rsa']['public']['n'],
)
final_data = (decrypted_data - a_x) % smc_data['a']['rsa']['public']['n']

{{'$data_{decrypted} = %d$' % decrypted_data}}   
{{'$data_{final} = %d$' % final_data}}  
{{f'$secret_a + secret_b + secret_c = {smc_data["a"]["secret"] + smc_data["b"]["secret"] +smc_data["c"]["secret"]}$'}}   